In [1]:
import os
import numpy as np
import pandas as pd
#from datetime import datetime, timedelta
import datetime
import xarray as xr
#from math import atan2, log
import sys
import uuid

sys.path.append('./../../salinity/subroutines/')
from read_routines import read_all_usv

#read in email
text_file = open('C:/Users/gentemann/Google Drive/f_drive/docs/projects/saildrone/baja/docs/email_for_file.txt', "r")
email_str = text_file.readlines()

dir_data_out = 'C:/Users/gentemann/Google Drive/public/2019_saildrone/felix/' #'f:/data/cruise_data/saildrone/saildrone_data/'
dir_data = 'C:/Users/gentemann/Google Drive/public/2019_saildrone/' #'f:/data/cruise_data/saildrone/saildrone_data/'
dir_data_pattern = 'C:/Users/gentemann/Google Drive/public/2019_saildrone/*.nc' 
adir_usv = dir_data_pattern


In [2]:
#input filename
dir_in='/home/ananda/jfpiolle/TMP/'
dir_out='/home/ananda/jfpiolle/TMP/'
filename_in='/home/ananda/jfpiolle/TMP/l3r_temp.nc'

#definitions and things you might have to set differently for each file
itow_mask1=45 #see code just a bit below for figure to determine where to set flags here
itow_mask2=-110 #see code just a bit below for figure to determine where to set flags here
ISDP = 'Saildrone'
SST_type = 'SSTdepth'
Annex_version = '01.1'
File_version = '01.0'
astr_platform='SD1002'
astr_title = 'Data from Saildrone cruise from SF to Guadalupe Island April-June 2018'
astr_uuid = str(uuid.uuid4())


In [3]:
data_dict = read_all_usv(dir_data_pattern)

number of file: 44
0 PMEL_Arctic_2015_sd126-ALL-1_min-v1
1 PMEL_Arctic_2015_sd128-ALL-1_min-v1
2 PMEL_Arctic_2016_sd126-ALL-1_min-v1
3 PMEL_Arctic_2016_sd128-ALL-1_min-v1
4 saildrone-gen_4-baja_2018-sd1002-20180411T180000-20180611T055959-1_minutes-v1
5 saildrone-gen_4-shark-2018-sd1001-20180315T000000-20180529T235959-1_minutes-v1.1581626958976
6 saildrone-gen_4-shark-2018-sd1004-20180315T000000-20180617T235959-1_minutes-v1.1581627077777
7 saildrone-gen_5-1021_atlantic-sd1021-20190525T000000-20191021T235959-1_minutes-v1.1571806429446_(1)
8 saildrone-gen_5-antarctica_circumnavigation_2019-sd1020-20190119T040000-20190803T043000-1_minutes-v1.1564884498845
9 saildrone-gen_5-arctic_misst_2019-sd1036-20190514T230000-20191011T183000-1_minutes-v1.1575336154680
10 saildrone-gen_5-arctic_misst_2019-sd1037-20190514T230000-20191011T183000-1_minutes-v1.1575487464625
11 saildrone-gen_5-atomic_eurec4a_2020-sd1026-20200117T000000-20200302T235959-1_minutes-v1.1589306725934
12 saildrone-gen_5-atomic_eure

In [10]:
for iname,name in enumerate(data_dict):
    if iname>3:
        continue
    ds = data_dict[name]
    indicative_date_time=(ds.time[0].dt.strftime("%Y%m%d%H%M%S")).data
    if ds.TEMP_CTD_MEAN.attrs.get('model_number'):
        Product_string = str(dataset.TEMP_CTD_MEAN.model_number) 
    else:
        Product_string = 'not_avail'
    if ds.TEMP_CTD_MEAN.attrs.get('serial_number'):
        Product_string = Product_string + '_' + str(dataset.TEMP_CTD_MEAN.serial_number) 
    else:
        Product_string = Product_string + '_' + 'not_avail'
        
    filename_L3R = dir_data_out + indicative_date_time + \
        '-' + ISDP + '-' + 'L3R' + '-' + SST_type + '-' +Product_string+ '-v' +Annex_version+ '-fv' +File_version+ '.nc'
    filename_L3R
print(ds)

<xarray.Dataset>
Dimensions:         (time: 149348, traj: 1)
Coordinates:
  * traj            (traj) float64 1.0
  * time            (time) datetime64[ns] 2016-05-23T00:01:15 ... 2016-09-03T18:02:15
Data variables:
    lat             (time) float64 53.84 53.84 53.84 53.84 ... 53.84 53.84 53.84
    lon             (time) float64 -166.6 -166.6 -166.6 ... -166.6 -166.6 -166.6
    TEMP_AIR_MEAN   (time) float64 ...
    RH_MEAN         (time) float64 ...
    BARO_PRES_MEAN  (time) float64 ...
    GUST_WND_MEAN   (time) float64 ...
    PAR_AIR_MEAN    (time) float64 ...
    SAL_CTD_MEAN    (time) float64 ...
    TEMP_CTD_MEAN   (time) float64 ...
    CDOM_MEAN       (time) float64 ...
    CHLOR_MEAN      (time) float64 ...
    UWND_MEAN       (time) float64 nan 1.024 nan nan nan ... nan nan nan nan nan
    VWND_MEAN       (time) float64 nan -1.902 nan nan nan ... nan nan nan nan
    wind_speed      (time) float64 nan 2.299 nan nan nan ... nan nan nan nan nan
    wind_direction  (time) float

In [6]:
#add global attributes that are missing
#some of these will need to be changed for new cruises
gstr = {'title': astr_title,
'summary':'none',
'references':'none',
'institution':'Saildrone',
'history':'Saildrone 6-hourly v1 files were used to create this file',
'comment':'none',
'license':'free and open',
'id':'SSTdepth',
'naming_authority':'org.shipborne-radiometer',
'product_version':'1.0',
'uuid':astr_uuid,
'l2r_version_id':'1.1', 
'netcdf_version_id':'4.6.1',
'date_created':datetime.datetime.now().strftime("%Y-%m-%dT%H:%M:%SZ"), #yyyy-mm-ddThh:mm:ssZ
'file_quality_level':3,
'spatial_resolution':'1 min',
'start_time':(ds.time[0].dt.strftime("%Y-%m-%dT%H:%M:%SZ")).data,
'time_coverage_start':(ds.time[0].dt.strftime("%Y-%m-%dT%H:%M:%SZ")).data,
'stop_time':(ds.time[-1].dt.strftime("%Y-%m-%dT%H:%M:%SZ")).data,
'time_coverage_end':(ds.time[-1].dt.strftime("%Y-%m-%dT%H:%M:%SZ")).data,
'northernmost_latitude':ds.lat.max(),
'geospatial_lat_max':ds.lat.max(),
'southernmost_latitude':ds.lat.min(),
'geospatial_lat_min':ds.lat.min(),
'easternmost_longitude':ds.lon.max(),
'geospatial_lon_max':ds.lon.max(),
'westernmost_longitude':ds.lon.min(),
'geospatial_lon_min':ds.lon.min(),
'geospatial_lat_units':'degrees_north',
'geospatial_lon_units':'degrees_east',
'source':'SSTdepth, wind_speed',
'platform':astr_platform,
'metadata_link':'TBD',
'keywords':'Oceans > Ocean Temperature > Sea Surface Temperature',
'keywords_vocabulary':'NASA Global Change Master Directory (GCMD) Science Keywords',
'acknowledgment':'The Schmidt Family Foundation, Saildrone, NASA Physical Oceanography',
'project':'International Shipborne Radiometer Network',
'publisher_name':'The ISRN Project Office',
'publisher_url':'http://www.shipborne.radiometer.org',
'publisher_email':email_str,
'processing_level':'1.0'}
#'sensor':str(dataset.TEMP_CTD_MEAN.sensor_description + '_' +\
#            dataset.TEMP_CTD_MEAN.model_number + '_' + dataset.TEMP_CTD_MEAN.serial_number + ', ' + \
#            dataset.UWND_MEAN.sensor_description + '_' + dataset.UWND_MEAN.model_number + '_' + \
#                    dataset.UWND_MEAN.serial_number)
#del 'nodc_template_version']

In [7]:
ds
#gattrs = ds.attrs.copy()
ds.attrs=gstr

In [9]:
ds

<xarray.Dataset>
Dimensions:                (time: 149348, traj: 1)
Coordinates:
  * time                   (time) datetime64[ns] 2016-05-23T00:01:15 ... 2016-09-03T18:02:15
  * traj                   (traj) float64 1.0
Data variables:
    lat                    (time) float64 53.84 53.84 53.84 ... 53.84 53.84
    lon                    (time) float64 -166.6 -166.6 -166.6 ... -166.6 -166.6
    sea_water_temperature  (time) float64 ...
    quality_level          (time) int8 2 2 2 2 2 2 2 2 2 2 ... 2 2 2 2 2 2 2 2 2
Attributes:
    title:                  Data from Saildrone cruise from SF to Guadalupe I...
    summary:                none
    references:             none
    institution:            Saildrone
    history:                Saildrone 6-hourly v1 files were used to create t...
    comment:                none
    license:                free and open
    id:                     SSTdepth
    naming_authority:       org.shipborne-radiometer
    product_version:        1.0
    uuid:                   35d06dfc-520f-478e-96a4-906015221d78
    l2r_version_id:         1.1
    netcdf_version_id:      4.6.1
    date_created:           2020-05-29T18:18:04Z
    file_quality_level:     3
    spatial_resolution:     1 min
    start_time:             2016-05-23T00:01:15Z
    time_coverage_start:    2016-05-23T00:01:15Z
    stop_time:              2016-09-03T18:02:15Z
    time_coverage_end:      2016-09-03T18:02:15Z
    northernmost_latitude:  <xarray.DataArray 'lat' ()>\narray(59.32477188)
    geospatial_lat_max:     <xarray.DataArray 'lat' ()>\narray(59.32477188)
    southernmost_latitude:  <xarray.DataArray 'lat' ()>\narray(53.84341812)
    geospatial_lat_min:     <xarray.DataArray 'lat' ()>\narray(53.84341812)
    easternmost_longitude:  <xarray.DataArray 'lon' ()>\narray(-163.96936035)
    geospatial_lon_max:     <xarray.DataArray 'lon' ()>\narray(-163.96936035)
    westernmost_longitude:  <xarray.DataArray 'lon' ()>\narray(-172.5045929)
    geospatial_lon_min:     <xarray.DataArray 'lon' ()>\narray(-172.5045929)
    geospatial_lat_units:   degrees_north
    geospatial_lon_units:   degrees_east
    source:                 SSTdepth, wind_speed
    platform:               SD1002
    metadata_link:          TBD
    keywords:               Oceans > Ocean Temperature > Sea Surface Temperature
    keywords_vocabulary:    NASA Global Change Master Directory (GCMD) Scienc...
    acknowledgment:         The Schmidt Family Foundation, Saildrone, NASA Ph...
    project:                International Shipborne Radiometer Network
    publisher_name:         The ISRN Project Office
    publisher_url:          http://www.shipborne.radiometer.org
    publisher_email:        ['info@shipborne-radiometer.org']
    processing_level:       1.0

In [8]:
# convert names
COORD_ATTR = 'lat lon time'
FLOAT_FILLVALUE = np.finfo(np.float32).min

swapvar = {
    'TEMP_CTD_MEAN': 'sea_water_temperature',
    'COG_MEAN': 'course_over_ground',
    'lat': 'lat',
    'lon': 'lon',
    'HDG_MEAN': 'true_bearing',
    'ROLL_MEAN': 'platform_roll',
    'PITCH_MEAN': 'platform_pitch',
    'SOG_MEAN': 'speed_over_ground',
    'TEMP_CTD_STDDEV':'sst_total_uncertainty'
}


ws_comment = 'Instrument located at to of Saildrone mast at 5.2 m' + '.  This was adjusted ' +\
            'to 10 m as ws_10m = ws*np.log(10./1e-4))/np.log(WS_height/1e-4'

ws_attr={'standard_name':'wind_speed',
         'long_name':'wind_speed',
         'valid_min':0,
         'valid_max':100,
         'units':'m s-1',
         '_FillValue':FLOAT_FILLVALUE,
         'source':'anemometer',
         'comment':ws_comment,
         'height':'10 m',"coordinates":COORD_ATTR}
wdir_attr={'standard_name':'wind_to_direction',
         'long_name':'local wind direction',
         'valid_min':0,
         'valid_max':360,
         'units':'degrees',
         '_FillValue':FLOAT_FILLVALUE,
         'source':'anemometer',
         'comment':ws_comment,
         'height':'10 m',"coordinates":COORD_ATTR}

# creates the new variables
# -------------------------
# 1. wind speed
WS=np.sqrt(ds.UWND_MEAN**2+ds.VWND_MEAN**2)
WS_height=int(5.2)  #5.2 m installed
WS_10m = (WS*np.log(10./1e-4))/np.log(WS_height/1e-4)
ds['wind_speed']=WS_10m
ds.wind_speed.fillna(FLOAT_FILLVALUE)
ds.wind_speed.attrs = ws_attr

ds['wind_direction']=np.arctan2(ds.VWND_MEAN, ds.UWND_MEAN)*np.pi/180.
ds.wind_direction.fillna(FLOAT_FILLVALUE)
ds.wind_direction.attrs=wdir_attr

# SWAP VARIABLE NAMES
# DROP VARIABLES NOT LISTED IN LIST_VAR
for var in ds:
    var2 = var
    print(var)
    if swapvar.get(var): 
        ds = ds.rename({var:swapvar.get(var)})
        var2 = swapvar.get(var)
#    if any(vv==var2 for vv in list_var):
#        ds #just a place holder does nothing
    else:
        ds = ds.drop(var2)     


# 4. Quality level
flag_bytes=np.byte((0,1,2,3,4,5)) #bytearray([0,1,2,3,4,5])
iqual_byte = np.ones(shape=ds.time.shape, dtype='b')*5  #change byte to b1
iqual_byte[:itow_mask1] = 2  #set at top of file from looking at data
iqual_byte[itow_mask2:] = 2
vattrs = {
    'long_name': 'measurement quality value',
    'flag_meanings': 'no_data bad_data worst_quality low_quality acceptable_quality best_quality',
    'flag_values': flag_bytes,
    '_FillValue': -128
}
ds['quality_level'] = xr.DataArray(
    iqual_byte,
    dims=['time'],
    attrs=vattrs
    )

l3r=ds
l3r.sea_water_temperature.attrs['valid_min']=260.0
l3r.sea_water_temperature.attrs['valid_max']=320.0
l3r.sea_water_temperature.attrs['units']='kelvin'
l3r.sea_water_temperature.attrs['long_name']='sea surface depth temperature at 0.6m'
l3r.time.attrs['standard_name']='time'
l3r.time.attrs['long_name']='time'
l3r.lon.attrs['standard_name']='longitude'
l3r.lon.attrs['long_name']='longitude'
l3r.lat.attrs['long_name']='latitude'
l3r.lat.attrs['standard_name']='latitude'
l3r.true_bearing.attrs['long_name']='platform true bearing'
l3r.true_bearing.attrs['standard_name']='platform_orientation'
l3r.speed_over_ground.attrs['long_name']='platform speed over ground'
l3r.sst_total_uncertainty.attrs['standard_name']='sea_water_temperature standard error'
l3r.sst_total_uncertainty.attrs['long_name']=' sea water temperature total uncertainty'
l3r.sst_total_uncertainty.attrs['valid_min']=0.0
l3r.sst_total_uncertainty.attrs['valid_max']=2.0
l3r.sst_total_uncertainty.attrs['units']='kelvin'
 
l3r.to_netcdf('example.nc')

lat
lon
TEMP_AIR_MEAN
RH_MEAN
BARO_PRES_MEAN
GUST_WND_MEAN
PAR_AIR_MEAN
SAL_CTD_MEAN
TEMP_CTD_MEAN
CDOM_MEAN
CHLOR_MEAN
UWND_MEAN
VWND_MEAN
wind_speed
wind_direction


AttributeError: 'Dataset' object has no attribute 'true_bearing'

In [91]:
    import xarray as xr
    import numpy as np
    from glob import glob
    import os
    
    #list names of variables to keep
    list_var = ['time','lat','lon','SOG_MEAN','COG_MEAN','HDG_MEAN','ROLL_MEAN','PITCH_MEAN',
                'UWND_MEAN','VWND_MEAN','WWND_MEAN','GUST_WND_MEAN','TEMP_AIR_MEAN','RH_MEAN','BARO_PRES_MEAN',
                'PAR_AIR_MEAN','TEMP_CTD_MEAN','SAL_CTD_MEAN','TEMP_RBR_MEAN','SAL_RBR_MEAN',
                'TEMP_O2_RBR_MEAN','CDOM_MEAN','CHLOR_MEAN','CHLOR_WETLABS_MEAN','WIND_MEASUREMENT_HEIGHT_MEAN']
    #list names of variables to swap to common names
    swapvar = {'TEMP_SBE37_MEAN':'TEMP_CTD_MEAN',
               'SAL_SBE37_MEAN':'SAL_CTD_MEAN',
               'SAL_MEAN':'SAL_CTD_MEAN',
               'TEMP_O2_RBR_MEAN':'TEMP_O2_MEAN',
               'TEMP_CTD_RBR_MEAN':'TEMP_RBR_MEAN',
               'CHLOR_RBR_MEAN':'CHLOR_MEAN',
               'SOG':'SOG_MEAN',
               'COG':'COG_MEAN',
               'HDG':'HDG_MEAN','HDG_FILTERED_MEAN':'HGD_MEAN',
               'ROLL_FILTERED_MEAN':'ROLL_MEAN','ROLL_FILTERD_MEAN':'ROLL_MEAN',
               'PITCH_FILTERED_MEAN':'PITCH_MEAN','PITCH':'PITCH_MEAN',
               'WIND_HEIGHT_MEAN':'WIND_MEASUREMENT_HEIGHT_MEAN'}
#'COG_FILTERED_MEAN':'COG_MEAN',
    #get list of all filenames in directory
    files = [x for x in glob(adir_usv)]
    print('number of file:',len(files))
    
    #go through each file, read in, normalize and put in dictionary with datasets
    for ifile,file in enumerate(files):
        #print(file)
        ds = xr.open_dataset(file)
        ds.close()
        if any(v=='latitude' for v in ds.coords.keys()):
            ds = ds.rename({'latitude':'lat','longitude':'lon'})
        if any(v=='latitude' for v in ds.dims.keys()):
            ds = ds.rename({'latitude':'lat','longitude':'lon'})
        if any(v=='latitude' for v in ds):
            ds = ds.rename({'latitude':'lat','longitude':'lon'})
        if any(v=='trajectory' for v in ds.dims.keys()):
            ds = ds.isel(trajectory=0)
    #    for v in ds.dims.keys():
        if any(v=='obs' for v in ds.dims.keys()):
            ds = ds.swap_dims({'obs':'time'})
        if any(v=='row' for v in ds.dims.keys()):
            ds = ds.swap_dims({'row':'time'})
        #remove any duplicates in time, keep only first value
        _, index = np.unique(ds['time'], return_index=True)
        ds=ds.isel(time=index)
        #renames some common variables to uniform name, drop variables not on list above
        if any(var=='wind_speed' for var in ds):
            #print(ds.wind_speed.attrs)
            #saildrone using meterological wind (blowind from) in early cruises
            #since noaa is calculating from uwnd and vwnd I went to 2019 arctic cruise 1037 and double checked values
            ds['UWND_MEAN']=-ds.wind_speed*np.sin(np.deg2rad(ds.wind_dir))
            ds['VWND_MEAN']=-ds.wind_speed*np.cos(np.deg2rad(ds.wind_dir))
#            ds.UWND_MEAN.attrs['units']=ds.wind_speed.attrs['units']
#            ds.VWND_MEAN.attrs['units']=ds.wind_speed.attrs['units']
            ds.UWND_MEAN.attrs = {'standard_name': 'eastward_wind', 'long_name': 'Eastward wind speed',
                                  'units': ds.wind_speed.attrs['units'], 'installed_height': '5.2'}
            ds.VWND_MEAN.attrs = {'standard_name': 'northward_wind', 'long_name': 'Northward wind speed',
                                  'units': ds.wind_speed.attrs['units'], 'installed_height': '5.2'}

        # SWAP VARIABLE NAMES
        # DROP VARIABLES NOT LISTED IN LIST_VAR
        for var in ds:
            var2 = var
            if swapvar.get(var): 
                if not var_exist(ds,var):
                    ds = ds.rename({var:swapvar.get(var)})
                    var2 = swapvar.get(var)
            if any(vv==var2 for vv in list_var):
                ds #just a place holder does nothing
            else:
                ds = ds.drop(var2)
                
        #check that there is a TEMP_CTD_MEAN, if not & temp_rbr_mean there, change it to temp_ctd_mean
        if any(var=='TEMP_CTD_MEAN' for var in ds):
            ds #just a place holder does nothing
        else:
            if any(var=='TEMP_RBR_MEAN' for var in ds):
                ds = ds.rename({'TEMP_RBR_MEAN':'TEMP_CTD_MEAN'})
        if any(var=='SAL_CTD_MEAN' for var in ds):
            ds #just a place holder does nothing
        else:
            if any(var=='SAL_RBR_MEAN' for var in ds):
                ds = ds.rename({'SAL_RBR_MEAN':'SAL_CTD_MEAN'})


number of file: 44


In [69]:
ds

<xarray.Dataset>
Dimensions:                         (time: 215731)
Coordinates:
    lat                             (time) float64 ...
    lon                             (time) float64 ...
  * time                            (time) datetime64[ns] 2019-05-14T23:00:00 ... 2019-10-11T18:30:00
    trajectory                      <U4 '1036'
Data variables:
    SOG_MEAN                        (time) float64 ...
    COG_MEAN                        (time) float64 ...
    COG_FILTERED_MEAN               (time) float64 ...
    COG_FILTERED_STDDEV             (time) float64 ...
    HDG                             (time) float64 ...
    HDG_FILTERED_MEAN               (time) float64 ...
    HDG_FILTERED_STDDEV             (time) float64 ...
    ROLL_FILTERED_MEAN              (time) float64 ...
    ROLL_FILTERED_STDDEV            (time) float64 ...
    ROLL_FILTERED_PEAK              (time) float64 ...
    PITCH_FILTERED_MEAN             (time) float64 ...
    PITCH_FILTERED_STDDEV           (time) float64 ...
    PITCH_FILTERED_PEAK             (time) float64 ...
    HDG_WING                        (time) float64 ...
    WING_HDG_FILTERED_MEAN          (time) float64 ...
    WING_HDG_FILTERED_STDDEV        (time) float64 ...
    WING_ROLL_FILTERED_MEAN         (time) float64 ...
    WING_ROLL_FILTERED_STDDEV       (time) float64 ...
    WING_ROLL_FILTERED_PEAK         (time) float64 ...
    WING_PITCH_FILTERED_MEAN        (time) float64 ...
    WING_PITCH_FILTERED_STDDEV      (time) float64 ...
    WING_PITCH_FILTERED_PEAK        (time) float64 ...
    WING_ANGLE                      (time) float64 ...
    UWND_MEAN                       (time) float64 ...
    UWND_STDDEV                     (time) float64 ...
    VWND_MEAN                       (time) float64 ...
    VWND_STDDEV                     (time) float64 ...
    WWND_MEAN                       (time) float64 ...
    WWND_STDDEV                     (time) float64 ...
    GUST_WND_MEAN                   (time) float64 ...
    GUST_WND_STDDEV                 (time) float64 ...
    WIND_HEIGHT_MEAN                (time) float64 ...
    WIND_HEIGHT_STDDEV              (time) float64 ...
    TEMP_AIR_MEAN                   (time) float64 ...
    TEMP_AIR_STDDEV                 (time) float64 ...
    RH_MEAN                         (time) float64 ...
    RH_STDDEV                       (time) float64 ...
    BARO_PRES_MEAN                  (time) float64 ...
    BARO_PRES_STDDEV                (time) float64 ...
    PAR_AIR_MEAN                    (time) float64 ...
    PAR_AIR_STDDEV                  (time) float64 ...
    TEMP_IR_SKY_HULL_MEAN           (time) float64 ...
    TEMP_IR_SKY_HULL_STDDEV         (time) float64 ...
    TEMP_IR_SEA_HULL_UNCOMP_MEAN    (time) float64 ...
    TEMP_IR_SEA_HULL_UNCOMP_STDDEV  (time) float64 ...
    TEMP_IR_SEA_WING_UNCOMP_MEAN    (time) float64 ...
    TEMP_IR_SEA_WING_UNCOMP_STDDEV  (time) float64 ...
    WAVE_DOMINANT_PERIOD            (time) float64 ...
    WAVE_SIGNIFICANT_HEIGHT         (time) float64 ...
    TEMP_SBE37_MEAN                 (time) float64 ...
    TEMP_SBE37_STDDEV               (time) float64 ...
    SAL_SBE37_MEAN                  (time) float64 ...
    SAL_SBE37_STDDEV                (time) float64 ...
    COND_SBE37_MEAN                 (time) float64 ...
    COND_SBE37_STDDEV               (time) float64 ...
    TEMP_CTD_RBR_MEAN               (time) float64 ...
    TEMP_CTD_RBR_STDDEV             (time) float64 ...
    SAL_RBR_MEAN                    (time) float64 ...
    SAL_RBR_STDDEV                  (time) float64 ...
    COND_RBR_MEAN                   (time) float64 ...
    COND_RBR_STDDEV                 (time) float64 ...
    O2_CONC_SBE37_MEAN              (time) float64 ...
    O2_CONC_SBE37_STDDEV            (time) float64 ...
    O2_SAT_SBE37_MEAN               (time) float64 ...
    O2_SAT_SBE37_STDDEV             (time) float64 ...
    O2_CONC_RBR_MEAN                (time) float64 ...
    O2_CONC_RB

In [ ]:
ds.get('WIND_MEASUREMENT_HEIGHT_MEAN')

In [ ]:
WIND_MEASUREMENT_HEIGHT_MEAN
WIND_HEIGHT_MEAN

In [4]:
file= 'saildrone-gen_5-antarctica_circumnavigation_2019-sd1020-20190119T040000-20190803T043000-1_minutes-v1.1564884498845.nc'
ds = xr.open_dataset('C:/Users/gentemann/Google Drive/public/2019_saildrone/'+file)

                     #ds = xr.open_dataset('C:/Users/gentemann/Google Drive/public/2019_saildrone\saildrone-gen_5-1021_atlantic-sd1021-20190525T000000-20191021T235959-1_minutes-v1.1571806429446 (1).nc')
#ds.WIND_HEIGHT_MEAN[0,:].plot()

<xarray.Dataset>
Dimensions:                        (obs: 248525, trajectory: 1)
Coordinates:
  * trajectory                     (trajectory) float32 1020.0
    time                           (trajectory, obs) datetime64[ns] ...
    latitude                       (trajectory, obs) float64 ...
    longitude                      (trajectory, obs) float64 ...
Dimensions without coordinates: obs
Data variables:
    SOG                            (trajectory, obs) float64 ...
    COG                            (trajectory, obs) float64 ...
    HDG                            (trajectory, obs) float64 ...
    HDG_WING                       (trajectory, obs) float64 ...
    ROLL                           (trajectory, obs) float64 ...
    PITCH                          (trajectory, obs) float64 ...
    WING_ANGLE                     (trajectory, obs) float64 ...
    BARO_PRES_MEAN                 (trajectory, obs) float64 ...
    BARO_PRES_STDDEV               (trajectory, obs) float64 ...
    TEMP_AIR_MEAN                  (trajectory, obs) float64 ...
    TEMP_AIR_STDDEV                (trajectory, obs) float64 ...
    RH_MEAN                        (trajectory, obs) float64 ...
    RH_STDDEV                      (trajectory, obs) float64 ...
    PAR_AIR_MEAN                   (trajectory, obs) float64 ...
    PAR_AIR_STDDEV                 (trajectory, obs) float64 ...
    TEMP_IR_UNCOR_MEAN             (trajectory, obs) float64 ...
    TEMP_IR_UNCOR_STDDEV           (trajectory, obs) float64 ...
    UWND_MEAN                      (trajectory, obs) float64 ...
    UWND_STDDEV                    (trajectory, obs) float64 ...
    VWND_MEAN                      (trajectory, obs) float64 ...
    VWND_STDDEV                    (trajectory, obs) float64 ...
    WWND_MEAN                      (trajectory, obs) float64 ...
    WWND_STDDEV                    (trajectory, obs) float64 ...
    GUST_WND_MEAN                  (trajectory, obs) float64 ...
    GUST_WND_STDDEV                (trajectory, obs) float64 ...
    TEMP_CTD_MEAN                  (trajectory, obs) float64 ...
    TEMP_CTD_STDDEV                (trajectory, obs) float64 ...
    COND_MEAN                      (trajectory, obs) float64 ...
    COND_STDDEV                    (trajectory, obs) float64 ...
    SAL_MEAN                       (trajectory, obs) float64 ...
    SAL_STDDEV                     (trajectory, obs) float64 ...
    O2_RBR_CONC_MEAN               (trajectory, obs) float64 ...
    O2_RBR_CONC_STDDEV             (trajectory, obs) float64 ...
    O2_RBR_SAT_MEAN                (trajectory, obs) float64 ...
    O2_RBR_SAT_STDDEV              (trajectory, obs) float64 ...
    TEMP_O2_RBR_MEAN               (trajectory, obs) float64 ...
    TEMP_O2_RBR_STDDEV             (trajectory, obs) float64 ...
    O2_AANDERAA_CONC_UNCOR_MEAN    (trajectory, obs) float64 ...
    O2_AANDERAA_CONC_UNCOR_STDDEV  (trajectory, obs) float64 ...
    O2_AANDERAA_SAT_MEAN           (trajectory, obs) float64 ...
    O2_AANDERAA_SAT_STDDEV         (trajectory, obs) float64 ...
    TEMP_O2_AANDERAA_MEAN          (trajectory, obs) float64 ...
    TEMP_O2_AANDERAA_STDDEV        (trajectory, obs) float64 ...
    CHLOR_MEAN                     (trajectory, obs) float64 ...
    CHLOR_STDDEV                   (trajectory, obs) float64 ...
Attributes:
    title:                     Antarctica Circumnavigation Surface Data
    license:                   2019 © SAILDRONE Inc.  All Rights Reserved.  T...
    wmo_id:                    4803902
    ncei_template_version:     NCEI_NetCDF_Trajectory_Template_v2.0
    Conventions:               CF-1.6, ACDD-1.3
    netcdf_version:            4.4.1.1
    featureType:               trajectory
    cdm_data_type:             Trajectory
    standard_name_vocabulary:  CF Standard Name Table v58
    description:               Saildrone NetCDF Format
    creator_name:              Saildrone
    platform:                  Saildrone
    institution:          

In [5]:
ds2 = ds.rename({'SOG':'SOG_MEAN'})

In [67]:
    import xarray as xr
    import numpy as np
    from glob import glob
    import os
    
    #list names of variables to keep
    list_var = ['time','lat','lon','SOG_MEAN','COG_MEAN','HDG_MEAN','ROLL_MEAN','PITCH_MEAN',
                'UWND_MEAN','VWND_MEAN','WWND_MEAN','GUST_WND_MEAN','TEMP_AIR_MEAN','RH_MEAN','BARO_PRES_MEAN',
                'PAR_AIR_MEAN','TEMP_CTD_MEAN','SAL_CTD_MEAN','TEMP_RBR_MEAN','SAL_RBR_MEAN',
                'TEMP_O2_RBR_MEAN','CDOM_MEAN','CHLOR_MEAN','CHLOR_WETLABS_MEAN','WIND_MEASUREMENT_HEIGHT_MEAN']
    #list names of variables to swap to common names
    swapvar = {'TEMP_SBE37_MEAN':'TEMP_CTD_MEAN',
               'SAL_SBE37_MEAN':'SAL_CTD_MEAN',
               'SAL_MEAN':'SAL_CTD_MEAN',
               'TEMP_O2_RBR_MEAN':'TEMP_O2_MEAN',
               'TEMP_CTD_RBR_MEAN':'TEMP_RBR_MEAN',
               'CHLOR_RBR_MEAN':'CHLOR_MEAN',
               'SOG':'SOG_MEAN',
               'COG':'COG_MEAN','COG_FILTERED_MEAN':'COG_MEAN',
               'HDG':'HDG_MEAN','HDG_FILTERED_MEAN':'HGD_MEAN',
               'ROLL_FILTERED_MEAN':'ROLL_MEAN','ROLL_FILTERD_MEAN':'ROLL_MEAN',
               'PITCH_FILTERED_MEAN':'PITCH_MEAN','PITCH':'PITCH_MEAN',
               'WIND_HEIGHT_MEAN':'WIND_MEASUREMENT_HEIGHT_MEAN'}

    #'SOG_FILTERED_MEAN':'SOG_MEAN',
    #get list of all filenames in directory
    files = [x for x in glob(adir_usv)]
    print('number of file:',len(files))
    
    #go through each file, read in, normalize and put in dictionary with datasets
    for ifile,file in enumerate(files):
        if ifile<8:
            continue
        #print(file)
        ds = xr.open_dataset(file)
        ds.close()
        if any(v=='latitude' for v in ds.coords.keys()):
            ds = ds.rename({'latitude':'lat','longitude':'lon'})
        if any(v=='latitude' for v in ds.dims.keys()):
            ds = ds.rename({'latitude':'lat','longitude':'lon'})
        if any(v=='latitude' for v in ds):
            ds = ds.rename({'latitude':'lat','longitude':'lon'})
        if any(v=='trajectory' for v in ds.dims.keys()):
            ds = ds.isel(trajectory=0)
    #    for v in ds.dims.keys():
        if any(v=='obs' for v in ds.dims.keys()):
            ds = ds.swap_dims({'obs':'time'})
        if any(v=='row' for v in ds.dims.keys()):
            ds = ds.swap_dims({'row':'time'})
        #remove any duplicates in time, keep only first value
        _, index = np.unique(ds['time'], return_index=True)
        ds=ds.isel(time=index)
        #renames some common variables to uniform name, drop variables not on list above
        if any(var=='wind_speed' for var in ds):
            #print(ds.wind_speed.attrs)
            #saildrone using meterological wind (blowind from) in early cruises
            #since noaa is calculating from uwnd and vwnd I went to 2019 arctic cruise 1037 and double checked values
            ds['UWND_MEAN']=-ds.wind_speed*np.sin(np.deg2rad(ds.wind_dir))
            ds['VWND_MEAN']=-ds.wind_speed*np.cos(np.deg2rad(ds.wind_dir))
#            ds.UWND_MEAN.attrs['units']=ds.wind_speed.attrs['units']
#            ds.VWND_MEAN.attrs['units']=ds.wind_speed.attrs['units']
            ds.UWND_MEAN.attrs = {'standard_name': 'eastward_wind', 'long_name': 'Eastward wind speed',
                                  'units': ds.wind_speed.attrs['units'], 'installed_height': '5.2'}
            ds.VWND_MEAN.attrs = {'standard_name': 'northward_wind', 'long_name': 'Northward wind speed',
                                  'units': ds.wind_speed.attrs['units'], 'installed_height': '5.2'}

        print(ds)
        # SWAP VARIABLE NAMES
        # DROP VARIABLES NOT LISTED IN LIST_VAR
        for var in ds:
            var2 = var
            if swapvar.get(var): 
                if !ds.get(var):
                    continue
                ds = ds.rename({var:swapvar.get(var)})
                var2 = swapvar.get(var)
            if any(vv==var2 for vv in list_var):
                ds #just a place holder does nothing
            else:
                ds = ds.drop(var2)
                
        #check that there is a TEMP_CTD_MEAN, if not & temp_rbr_mean there, change it to temp_ctd_mean
        if any(var=='TEMP_CTD_MEAN' for var in ds):
            ds #just a place holder does nothing
        else:
            if any(var=='TEMP_RBR_MEAN' for var in ds):
                ds = ds.rename({'TEMP_RBR_MEAN':'TEMP_CTD_MEAN'})
        if any(var=='SAL_CTD_MEAN' for var in ds):
            ds #just a place holder does nothing
        else:
            if any(var=='SAL_RBR_MEAN' for var in ds):
                ds = ds.rename({'SAL_RBR_MEAN':'SAL_CTD_MEAN'})


SyntaxError: invalid syntax (<ipython-input-67-ae392c198f6f>, line 73)

In [42]:
if np.all(ds.variables=='SOG_MEAN'):
    print('y')

In [57]:
np.any(ds.keys()=='COG')

False

In [88]:


var_exist(ds,'SOG_MEAN')

True